In [ ]:
#run this file on google colab
#change hardware accelerator to GPU to reduce computation(model training time)
#steps to change hardware accelerator 
#On taskbar, Runtime -> Change runtime type -> Hardware accelerator(dropdown) -> GPU -> Save

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/Gestures

In [ ]:
#importing libraries
import os 
import cv2
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical


In [ ]:
#selecting gestures for the model
images = []
gesture_names = []
gesture_counts = []


while(True):
  folder_name = input("ENTER GESTURE FOLDER NAME : ")
  if folder_name == 'exit':
    break

  gesture_name = input("ENTER GESTURE NAME : ")
  gesture_names.append(gesture_name)

  gesture_count = int(input("Number of "+gesture_name + " : "))
  gesture_counts.append(gesture_count)

  image_count = 0
  
  for i in os.listdir('./'+folder_name):
    image = cv2.imread(folder_name + '/' + i , 0)
    images.append(image)
    print(image_count , i)
    image_count += 1

    if image_count == gesture_count:
      break

In [ ]:
x = np.array(images)
x = x.reshape(x.shape[0],180,180,1)

gesture_Y = []
count_y = 0
for i in range(len(gesture_names)):
  for i in range(gesture_counts[count_y]):
    gesture_Y.append(count_y)
  count_y += 1 

y = np.array(gesture_Y)
y = to_categorical(y)

x_train , x_test , y_train , y_test = train_test_split(x,y,test_size = 0.25)

In [ ]:
def train_model(epochs,batch_size,activation,optimizer,output_count):

  batch_size = batch_size
  epochs = epochs
  
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),activation=activation,input_shape=(180,180,1)))
  model.add(Conv2D(64, (3, 3), activation=activation))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(128, activation=activation))
  model.add(Dropout(0.5))
  model.add(Dense(output_count, activation='softmax'))

  model.compile(loss=keras.losses.categorical_crossentropy,optimizer=optimizer,metrics=['accuracy'])
  model_re = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))

  return model_re.history['accuracy'][-1],model_re.history['loss'][-1],model_re.history['val_accuracy'][-1],model_re.history['val_loss'][-1]

In [ ]:
#model performance of various combination
optimizers = ['Adadelta','Adagrad','Adam','SGD']
activation = ['relu','sigmoid','tanh','linear']

total = []
for opt in optimizers:
  for act in activation:
    row = []
    row.append(opt)
    row.append(act)
    out = train_model(5,25,act,opt,3)
    for res in out:
      row.append(res)
    total.append(row)

In [ ]:
#best model
train_model(5,25,'relu','Adagrad',3)

In [ ]:
#downloading model
from keras.model import load_model
model.save("25al_gestures.h5")